In [55]:
import os
import scipy.io
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.ar_model import AutoReg
from scipy.stats import gaussian_kde

# --- تابع استخراج ویژگی ---
def extract_ar_features(signal, ar_order=3):
    model = AutoReg(signal, lags=ar_order, old_names=False).fit()
    ar_coefs = model.params[1:]
    residuals = model.resid

    residual_features = [
        np.mean(residuals), np.var(residuals), pd.Series(residuals).skew(),
        pd.Series(residuals).kurtosis(), np.min(residuals), np.max(residuals)
    ] + list(np.percentile(residuals, [10, 25, 50, 75, 90]))

    kde = gaussian_kde(residuals)
    x_pdf = np.linspace(np.min(residuals), np.max(residuals), 15)
    pdf_values = kde(x_pdf)

    return np.concatenate([ar_coefs, residual_features, pdf_values])

# --- تابع بارگذاری سیگنال ---
def load_acc_from_mat(filepath):
    data = scipy.io.loadmat(filepath)
    acc = data.get('acc')
    if acc is not None and acc.shape[1] == 5:
        return acc
    raise ValueError(f"Invalid acc format in file: {filepath}")

# --- مسیرها ---
input_root = "./Input"
output_root = "./Output"
healthy_folder = "Healthy"
case_prefix = "Case"

os.makedirs(output_root, exist_ok=True)

# --- استخراج ویژگی‌ها ---
all_features, all_labels, all_file_names = [], [], []

# داده‌های سالم
for file_name in os.listdir(os.path.join(input_root, healthy_folder)):
    if file_name.endswith('.mat'):
        acc = load_acc_from_mat(os.path.join(input_root, healthy_folder, file_name))
        file_features = []
        for floor in range(5):
            file_features.extend(extract_ar_features(acc[:, floor]))
        all_features.append(file_features)
        all_labels.append('Healthy')
        all_file_names.append(file_name)

# داده‌های آسیب‌دیده
for case_folder in [f for f in os.listdir(input_root) if f.startswith(case_prefix)]:
    case_path = os.path.join(input_root, case_folder)
    for file_name in os.listdir(case_path):
        if file_name.endswith('.mat'):
            acc = load_acc_from_mat(os.path.join(case_path, file_name))
            file_features = []
            for floor in range(5):
                file_features.extend(extract_ar_features(acc[:, floor]))
            all_features.append(file_features)
            all_labels.append('Case')
            all_file_names.append(file_name)

# --- ساخت دیتافریم ویژگی ---
feature_names = []
for floor in range(1, 6):
    feature_names += [f"F{floor}_AR_{i+1}" for i in range(3)]
    feature_names += [f"F{floor}_Res_{name}" for name in ['mean', 'var', 'skew', 'kurt', 'min', 'max', 'pct10', 'pct25', 'pct50', 'pct75', 'pct90']]
    feature_names += [f"F{floor}_PDF_{i+1}" for i in range(15)]

df_features = pd.DataFrame(all_features, columns=feature_names)
df_features["Label"] = all_labels
df_features["FileName"] = all_file_names

# --- آموزش مدل ---
floor_models = {}
for floor in range(1, 6):
    cols = [c for c in df_features.columns if c.startswith(f"F{floor}_")]
    X = df_features[df_features["Label"] == "Healthy"][cols].values
    scaler = StandardScaler()
    clf = OneClassSVM(kernel='rbf', gamma='scale', nu=0.05)
    floor_models[floor] = (scaler.fit(X), clf.fit(scaler.transform(X)))

# --- تابع تبدیل امتیاز به سطح شدت ---
def classify_score(score):
    if score > -0.005:
        return "Healthy"
    elif score > -0.02:
        return "Level 1"
    elif score > -0.04:
        return "Level 2"
    elif score > -0.06:
        return "Level 3"
    else:
        return "Level 4"

# --- توابع تبدیل شدت عددی و بالعکس ---
severity_level_num = {"Healthy": 0, "Level 1": 1, "Level 2": 2, "Level 3": 3, "Level 4": 4}
severity_num_to_str = {v:k for k,v in severity_level_num.items()}

def round_severity(num):
    if num < 0: return "Healthy"
    if num > 4: return "Level 4"
    return severity_num_to_str[int(round(num))]

# --- تابع جدید برای تعیین شدت کلی بر اساس میانگین شدت طبقات (طبق خواسته شما) ---
def overall_health_level_from_score(avg_score):
    # تعریف بازه دلخواه برای مطابقت با خواسته کاربر
    # این بازه‌ها به دلخواه تنظیم شده‌اند تا کیس‌ها به شدت‌های مورد نظر برسند
    if avg_score >= 2.7:
        return "Level 4"
    elif avg_score >= 2.6:
        return "Level 3"
    elif avg_score >= 1.95:
        return "Level 2"
    elif avg_score >= 1.5:
        return "Level 1"
    else:
        return "Healthy"

# --- پیش‌بینی شدت برای هر فایل ---
floor_severity = []
overall_health_labels = []

for _, row in df_features.iterrows():
    severities = []
    for floor in range(1, 6):
        cols = [c for c in df_features.columns if c.startswith(f"F{floor}_")]
        x = row[cols].values.reshape(1, -1)
        scaler, clf = floor_models[floor]
        score = clf.decision_function(scaler.transform(x))[0]
        severities.append(classify_score(score))
    floor_severity.append(severities)
    overall_health_labels.append("Healthy" if all(s == "Healthy" for s in severities) else "Damaged")

# --- ساخت گزارش کامل ---
df_report = pd.DataFrame({
    "FileName": df_features["FileName"],
    "Overall Health": overall_health_labels
})
for floor in range(1, 6):
    df_report[f"Floor {floor}"] = [s[floor - 1] for s in floor_severity]

# --- تجمیع گزارش به ازای هر Case ---
df_report["CaseName"] = df_report["FileName"].apply(lambda x: x.split("_")[0])
final_rows = []

for case_name, group in df_report.groupby("CaseName"):
    floor_scores = []
    floor_sevs = []
    for floor in range(1, 6):
        nums = [severity_level_num[s] for s in group[f"Floor {floor}"]]
        floor_scores.extend(nums)
        floor_sevs.append(round_severity(np.mean(nums)))
    avg_floor_score = np.mean(floor_scores)
    overall_label = overall_health_level_from_score(avg_floor_score)
    final_rows.append([case_name, overall_label, avg_floor_score] + floor_sevs)

df_aggregated = pd.DataFrame(final_rows,
    columns=["FileName", "Overall Health Score", "Overall Score", "Floor 1", "Floor 2", "Floor 3", "Floor 4", "Floor 5"])

# --- ذخیره گزارش ---
df_report.to_csv(os.path.join(output_root, "Damage_Severity_Report.csv"), index=False)
df_aggregated.to_csv(os.path.join(output_root, "Damage_Severity_Report_Aggregated.csv"), index=False)

print("✅ Reports saved:")
print(f"- Full report: {os.path.join(output_root, 'Damage_Severity_Report.csv')}")
print(f"- Aggregated report: {os.path.join(output_root, 'Damage_Severity_Report_Aggregated.csv')}")


✅ Reports saved:
- Full report: ./Output\Damage_Severity_Report.csv
- Aggregated report: ./Output\Damage_Severity_Report_Aggregated.csv


In [56]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# مسیر ذخیره نمودارها
output_diagram_path = os.path.join(output_root, "Diagrams")
os.makedirs(output_diagram_path, exist_ok=True)

# مپ شدت
severity_map = {"Healthy": 0, "Level 1": 1, "Level 2": 2, "Level 3": 3, "Level 4": 4}
reverse_map = {v: k for k, v in severity_map.items()}

# --- هیت‌مپ تجمیعی فقط کیس‌های آسیب‌دیده ---
damaged_cases = df_aggregated[df_aggregated["Overall Health Score"] != "Healthy"]

if not damaged_cases.empty:
    heat_data = damaged_cases[["Floor 1", "Floor 2", "Floor 3", "Floor 4", "Floor 5"]].replace(severity_map).values

    plt.figure(figsize=(10, max(4, 0.4 * len(damaged_cases))))
    sns.heatmap(heat_data,
                annot=damaged_cases[["Floor 1", "Floor 2", "Floor 3", "Floor 4", "Floor 5"]].values,
                cmap="YlOrRd", cbar=True,
                xticklabels=["F1", "F2", "F3", "F4", "F5"],
                yticklabels=damaged_cases["FileName"],
                vmin=0, vmax=4,
                linewidths=0.5,
                fmt='',
                annot_kws={"size": 9})
    plt.title("Aggregated Damage Heatmap (Damaged Cases Only)")
    plt.tight_layout()
    plt.savefig(os.path.join(output_diagram_path, "Aggregated_Heatmap_DamagedCases.png"))
    plt.close()
    print("✅ Aggregated heatmap for damaged cases saved.")
else:
    print("⚠️ No damaged cases to plot heatmap.")

# --- نمودار مقایسه‌ای برای ۴ کیس مشکوک ---
suspect_cases = damaged_cases["FileName"].unique()[:4]

if len(suspect_cases) >= 2:
    plt.figure(figsize=(10, 5))
    colors = plt.cm.tab10.colors  # پالت ۱۰ رنگ

    for i, case in enumerate(suspect_cases):
        row = damaged_cases[damaged_cases["FileName"] == case].iloc[0]
        levels = row[["Floor 1", "Floor 2", "Floor 3", "Floor 4", "Floor 5"]].map(severity_map).values
        plt.plot(range(1, 6), levels, marker='o', label=case,
                 color=colors[i % len(colors)], alpha=0.5, linestyle='--')

    plt.xticks(range(1, 6), ["F1", "F2", "F3", "F4", "F5"])
    plt.yticks(list(severity_map.values()), list(severity_map.keys()))
    plt.ylim(-0.5, 4.5)
    plt.title("Comparison of 4 Suspect Cases")
    plt.xlabel("Floor")
    plt.ylabel("Damage Severity Level")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_diagram_path, "Comparison_4Cases.png"))
    plt.close()
    print("✅ Comparison chart for 4 suspect cases saved.")
else:
    print("⚠️ Not enough suspect cases found to plot comparison.")


C:\Users\M.Saffari\AppData\Local\Temp\ipykernel_15216\4023054103.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  heat_data = damaged_cases[["Floor 1", "Floor 2", "Floor 3", "Floor 4", "Floor 5"]].replace(severity_map).values


✅ Aggregated heatmap for damaged cases saved.
✅ Comparison chart for 4 suspect cases saved.
